# Exploratory Data Analysis (EDA)

Introduction <a name="introduction"></a>

The goal of this notebook is to ensure the quality and consistency of the data loaded into the database.  
We will check for:
- Duplicate records in all tables
- Foreign key integrity between related tables
- Price consistency between `Producto` and `DetalleOrden`
- Distribution of order and payment statuses

## Table of Contents

1. [Introduction](#introduction)
2. [Duplicate Check](#duplicate-check)
3. [Foreign Key Consistency](#foreign-key-consistency)
    - [User Foreign Keys](#user-foreign-keys)
    - [Product Foreign Keys](#product-foreign-keys)
4. [Price Consistency](#price-consistency)
5. [Order and Payment Status Distribution](#order-and-payment-status-distribution)
6. [Conclusions](#conclusions)

In [ ]:
from database.database_connection import get_session
from database.models import (
    Usuario, Categoria, Producto, Orden, DetalleOrden,
    DireccionEnvio, Carrito, MetodoPago, OrdenMetodoPago,
    ResenaProducto, HistorialPago
)
from sqlalchemy import select, func
from EDA_helper import find_invalid_user_references, find_invalid_product_references, check_duplicates

In [2]:

# Variable que vamos a usar durante distintos chequeos.
tables = [Usuario,
    Categoria,
    Producto,
    Orden,
    DetalleOrden,
    DireccionEnvio,
    Carrito,
    MetodoPago,
    OrdenMetodoPago,
    ResenaProducto,
    HistorialPago]

In [3]:
session = get_session()

## 1. Duplicate Check <a name="duplicate-check"></a>

We verify that there are no duplicate records in any of the main tables.


In [4]:
check_duplicates(session, tables)

Quantity of repeated values in table Usuario : 0
Quantity of repeated values in table Categoria : 0
Quantity of repeated values in table Producto : 0
Quantity of repeated values in table Orden : 0
Quantity of repeated values in table DetalleOrden : 0
Quantity of repeated values in table DireccionEnvio : 0
Quantity of repeated values in table Carrito : 0
Quantity of repeated values in table MetodoPago : 0
Quantity of repeated values in table OrdenMetodoPago : 0
Quantity of repeated values in table ResenaProducto : 0
Quantity of repeated values in table HistorialPago : 0


Los datos muestran que no hay valores repetidos en nuestra data

## 2. Foreign Key Consistency <a name="foreign-key-consistency"></a>

### 2.1 User Foreign Keys <a name="user-foreign-keys"></a>

We check that all foreign keys referencing users actually point to existing users in the `Usuario` table.

---

In [ ]:
resultados = find_invalid_user_references(session)
for tabla, usuarios in resultados.items():
    print(f"{tabla}: {usuarios}")

ordenes: []
carrito: []
direcciones_envio: []
resenas: []


La lista vacia significa que no hay ningun carrito sin usuario valido.

### 2.2 Product Foreign Keys <a name="product-foreign-keys"></a>

We check that all foreign keys referencing products actually point to existing products in the `Producto` table.

---

In [ ]:
resultados = find_invalid_product_references(session)
for tabla, productos in resultados.items():
    print(f"{tabla}: {productos}")

detalle_ordenes: []
carrito: []
resenas: []


## 3. Price Consistency <a name="price-consistency"></a>

We compare the `precio_unitario` in `DetalleOrden` with the `precio` in `Producto` to detect any inconsistencies.

---

In [7]:
stmt = (
    select(func.count())
    .select_from(DetalleOrden)            # aclaramos tabla base
    .join(DetalleOrden.producto)          # join por relación ORM
    .where(DetalleOrden.precio_unitario == Producto.precio)
)

result = session.execute(stmt).scalar_one()

print(f"Número de coincidencias precio_unitario == precio: {result}")

Número de coincidencias precio_unitario == precio: 0


Como podemos ver, en ningun item de toda la lista, es congruente el precio que a parece en la tabla Producto, con la tabla DetalleOrden que tambien lleva el precio de cada producto.

## 4. Dimension Data Distribution <a name="order-and-payment-status-distribution"></a>

We analyze and visualize the distribution of order statuses and payment statuses.

---

In [8]:
total_ordenes = session.execute(select(func.count(Orden.orden_id))).scalar_one()
stmt = (
    select(Orden.estado, func.count().label("cantidad"))
    .group_by(Orden.estado)
    .order_by(func.count().desc())
)

result = session.execute(stmt).all()

for estado, cantidad in result:
    print(f"Cantidad de órdenes en estado {estado}: {cantidad} ({cantidad / total_ordenes * 100:.2f}%)")

Cantidad de órdenes en estado Cancelado: 2510 (25.10%)
Cantidad de órdenes en estado Completado: 2498 (24.98%)
Cantidad de órdenes en estado Enviado: 2498 (24.98%)
Cantidad de órdenes en estado Pendiente: 2494 (24.94%)


In [9]:
total_historial_pagos = session.execute(select(func.count(HistorialPago.orden_id))).scalar_one()
stmt = (
    select(HistorialPago.estado_pago, func.count().label("cantidad"))
    .group_by(HistorialPago.estado_pago)
    .order_by(func.count().desc())
)

result = session.execute(stmt).all()

for estado, cantidad in result:
    print(f"Cantidad de órdenes en estado {estado}: {cantidad} ({cantidad / total_ordenes * 100:.2f}%)")

Cantidad de órdenes en estado Fallido: 2546 (25.46%)
Cantidad de órdenes en estado Procesando: 2542 (25.42%)
Cantidad de órdenes en estado Pagado: 2457 (24.57%)
Cantidad de órdenes en estado Reembolsado: 2455 (24.55%)


## Conclusions <a name="conclusions"></a>

- No duplicate values found in the tables.
- All carts are linked to existing users.
- All orders are linked to existing products.
- No matches between `DetalleOrden.precio_unitario` and `Producto.precio`, which may indicate data inconsistencies.
- No noise detected in the `HistorialPago` or `Orden` tables; both have a uniform data distribution.

---